In [1]:
import cv2
import mediapipe as mp
import pandas as pd
import re
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_holistic = mp.solutions.holistic

df = pd.DataFrame()
frames = []

In [2]:
# Regular expression pattern to extract coordinates
pattern = r'x: ([-+]?[0-9]*\.?[0-9]+)\ny: ([-+]?[0-9]*\.?[0-9]+)\nz: ([-+]?[0-9]*\.?[0-9]+)'

def get_coordinates(input_string):
    # Extract coordinates using regex
    matches = re.findall(pattern, input_string)

    # Convert the extracted coordinates to a list of dictionaries
    coordinates = [{'x': float(match[0]), 'y': float(match[1]), 'z': float(match[2])} for match in matches]
    return coordinates

In [3]:
import os
path = os.getcwd()

In [4]:
def get_video_speed_factor(input_path, target_duration=1.0):
    # Open the input video
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the current duration of the video
    video_duration = frame_count / fps

    # Calculate the speed factor needed to make the video last approximately 1 second
    speed_factor = video_duration / target_duration

    # Release video capture
    cap.release()

    return speed_factor

In [5]:
def speed_up_video_larger(input_path, output_path, speed_factor):
    # Import video clip
    clip = VideoFileClip(input_path)
    # Modify the FPS
    clip = clip.set_fps(clip.fps * speed_factor)
    # Apply speed up
    final = clip.fx(vfx.speedx, speed_factor)
    # Save video clip
    final.write_videofile(output_path)

In [6]:

def resize_video(input_path, output_path, target_width, target_height):
    # Open the input video
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define codec and create VideoWriter object for the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Resize the frame to the target width and height
        resized_frame = cv2.resize(frame, (target_width, target_height))

        # Write the resized frame to the output video
        out.write(resized_frame)

    # Release video capture and writer
    cap.release()
    out.release()

In [7]:
def change_fps(input_path, output_path, target_fps):
    if target_fps <= 0:
        raise ValueError("New FPS must be greater than zero.")

    # Open the input video file
    cap = cv2.VideoCapture(input_path)

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    # Calculate the new interval between frames based on the desired FPS
    interval = round(fps / target_fps)

    # Create a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'XVID') # You may need to change the codec based on your system
    out = cv2.VideoWriter(output_path, fourcc, target_fps, (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Write the frame to the output video
        if frame_count % interval == 0:
            out.write(frame)

        frame_count += 1

    # Release video objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [8]:
def extract_video(video, target, sequence_id, target_width, target_height):
    global frames
    # For webcam input:
    cap = cv2.VideoCapture(video)
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the current duration of the video
    video_duration = frame_count / fps

    print('-' * 20)
    print("FPS: ", fps, " VIDEO DURATION: ", video_duration)
    print('-' * 20)

    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the BGR frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame with the hand tracking model
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            row_data = {
                "sequence_id": sequence_id,
                "target": target
            }
            detected_hands = []
            for idx, landmarks in enumerate(results.multi_hand_landmarks):
                # Iterate through detected hand landmarks
                for landmark_idx, landmark in enumerate(landmarks.landmark):
                    x, y = landmark.x * frame.shape[1], landmark.y * frame.shape[0]
                    
                    # Determine hand side based on the index of the hand in the list
                    if idx == 0:
                        hand_side = "Left"
                    else:
                        hand_side = "Right"
                    
                    row_data[f'x_{hand_side}_hand_{landmark_idx}'] =  x / target_width
                    row_data[f'y_{hand_side}_hand_{landmark_idx}'] =  y / target_height

                    detected_hands.append(hand_side)

                    # Draw circles on the frame
                    cv2.circle(frame, (int(x), int(y)), 5, (0, 255, 0), -1)
                    cv2.putText(frame, hand_side, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            frames.append(row_data)
        cv2.imshow('Hand Tracking', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [9]:
import glob, os
import time
os.chdir("D:\Documents\Trabajo\megaproyecto_model\data/asl_videos")

sequence_id = 0
for file in glob.glob("*.mp4"):
    sequence_id += 1
    name = " ".join(file.split(" ")[:-1])

    input_path = f'D:\Documents\Trabajo\megaproyecto_model\data/asl_videos/{file}'


    output_path_r = 'D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/resized_output.mp4'
    output_path_f = 'D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/fps_output.mp4'
    output_path_s = 'D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4'

    target_width = 640
    target_height = 360
    desired_fps = 30

    speed_up_video_larger(input_path, output_path_s, speed_factor)
    resize_video(output_path_s, output_path_r, target_width, target_height)
    change_fps(output_path_r, output_path_f, desired_fps)

    extract_video(output_path_f, name, sequence_id, target_width, target_height)

Moviepy - Building video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4.
MoviePy - Writing audio in speed_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4



Moviepy - Done !
Moviepy - video ready D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4
--------------------
FPS:  30  VIDEO DURATION:  1.2666666666666666
--------------------
Moviepy - Building video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4.
MoviePy - Writing audio in speed_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4



Moviepy - Done !
Moviepy - video ready D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4
--------------------
FPS:  30  VIDEO DURATION:  1.2333333333333334
--------------------
Moviepy - Building video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4.
MoviePy - Writing audio in speed_outputTEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4
--------------------
FPS:  30  VIDEO DURATION:  1.0
--------------------
Moviepy - Building video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4.
MoviePy - Writing audio in speed_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4



Moviepy - Done !
Moviepy - video ready D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4
--------------------
FPS:  30  VIDEO DURATION:  1.0
--------------------
Moviepy - Building video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4.
MoviePy - Writing audio in speed_outputTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4



Moviepy - Done !
Moviepy - video ready D:\Documents\Trabajo\megaproyecto_model\data/asl_preprocessing/speed_output.mp4
--------------------
FPS:  30  VIDEO DURATION:  1.2
--------------------


In [10]:
df = pd.DataFrame(frames)

In [11]:
df['sequence_id'] = df['sequence_id'].astype(int)

In [12]:
len(df)

8950

In [13]:
df.head()

,sequence_id,target,x_Left_hand_0,y_Left_hand_0,x_Left_hand_1,y_Left_hand_1,x_Left_hand_2,y_Left_hand_2,x_Left_hand_3,y_Left_hand_3,...,x_Right_hand_16,y_Right_hand_16,x_Right_hand_17,y_Right_hand_17,x_Right_hand_18,y_Right_hand_18,x_Right_hand_19,y_Right_hand_19,x_Right_hand_20,y_Right_hand_20
0,1,bathroom,0.265447,0.474239,0.290252,0.441311,0.305287,0.405506,0.309431,0.372621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,bathroom,0.261486,0.448284,0.283510,0.412658,0.292817,0.370035,0.290641,0.334303,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,bathroom,0.260146,0.429222,0.277865,0.388216,0.282158,0.344853,0.276773,0.310717,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,bathroom,0.262401,0.420721,0.274276,0.374675,0.273511,0.331812,0.265531,0.300877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,bathroom,0.268579,0.414869,0.273348,0.369737,0.266685,0.331563,0.256571,0.304999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
dir_data = "D:\Documents\Trabajo\megaproyecto_model" + "/data.csv"
df.to_csv(dir_data, index=False)

In [15]:
validation_path = "D:\Documents\Trabajo\megaproyecto_model" + "/validation.csv"

import random
grouped = df.groupby('target')
selected_validation_ids = []
for group_name, group_indices in grouped.groups.items():
    ids = list(set(df.loc[group_indices]['sequence_id'].values))
    if (len(ids) < 0):
        raise ("Error")
    selected_validation_ids.append(random.choice(ids))

validation_df = df[df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
validation_df.to_csv(validation_path, index=False)

In [16]:
train_path = "D:\Documents\Trabajo\megaproyecto_model" + "/train.csv"
train_df = df[~df['sequence_id'].isin(selected_validation_ids)][["sequence_id", "target"]]
train_df.to_csv(train_path, index=False)